#  Practica de limpieza de texto con pandas - Solución

#### Por Jose R. Zapata - https://joserzapata.github.io/


> **Objetivo**: practicar métodos de `pandas.Series.str` para limpiar y normalizar texto en DataFrames.
> Cada ejercicio incluye un **Enunciado** y una celda donde se ve el resultado de la solución.



**Referencias:**
- Procesamiento básico de texto (curso de NLP): https://joserzapata.github.io/courses/nlp/procesamiento-basico/
- Documentación de `pandas.Series.str`: https://pandas.pydata.org/docs/reference/series.html#string-handling


## Instalación de dependencias

Instala e importa las librerias que necesites para ejecutar los ejercicios. Ejemplos de comandos de instalación (usa el entorno virtual del proyecto):

```bash
# activar el entorno virtual (si no está activado):
source .venv/bin/activate
# instalar pandas y Jupyter Notebook / JupyterLab
pip install pandas
pip install notebook  # Jupyter Notebook (legacy)
pip install jupyterlab  # alternativa moderna (opcional)
```

Si necesitas otras librerías comunes para NLP, por ejemplo:

```bash
pip install scikit-learn nltk spacy regex
```

A continuación hay una celda de código de comprobación que importa `pandas` y `notebook`, muestra sus versiones y crea un DataFrame de ejemplo. Puedes ejecutar esa celda para verificar la instalación.

In [4]:
# Comprobación de instalaciones: muestra versiones y un DataFrame de ejemplo
# ----------------------------------------------------
import importlib
import sys

for pkg in ("pandas", "notebook"):
    try:
        m = importlib.import_module(pkg)
        print(f"{pkg} version:", getattr(m, "__version__", "unknown"))
    except Exception as e:
        print(f"{pkg} NOT INSTALLED or failed to import: {e}")

# Pequeño chequeo con pandas
try:
    import pandas as pd

    df = pd.DataFrame({"a": [1, 2, 3], "b": ["x", "y", "z"]})
    print("\nDataFrame de ejemplo:")  # ← aquí se corrigió el salto de línea con \n
    print(df)
except Exception as e:
    print("Pandas import failed:", type(e).__name__, e)


pandas version: 2.3.3
notebook version: 7.4.7

DataFrame de ejemplo:
   a  b
0  1  x
1  2  y
2  3  z


In [5]:
# Copie su código aca
import pandas as pd
import notebook
import numpy as np


print("pandas:", pd.__version__)
print("notebook:", getattr(notebook, "__version__", "unknown"))
print("numpy:", pd.__version__)

pandas: 2.3.3
notebook: 7.4.7
numpy: 2.3.3


In [7]:
import pandas as pd

## Conjuntos de datos de juguete
Creamos 4 DataFrames para practicar distintas tareas de limpieza: texto general, tweets, productos y personas.

In [8]:
## Texto general
df_texto = pd.DataFrame(
    {
        "texto": [
            "  ¡Hola Mundo!!!  Esto es   un EJEMPLO: visita https://miweb.com  #DataScience  :)  ",
            "Pandas > numpy? 🤔  Email: persona@example.com   ",
            "Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan",
            "Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123",
            "      TABLAS\t, \n espacios   y saltos de línea.\r\n",
            "Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44",
            "Dirección: Cll 10 # 5-20; Medellín. Barrio: La América",
            "Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",
        ]
    }
)

## Tweets
df_tweets = pd.DataFrame(
    {
        "tweet": [
            "RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python",
            "¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊",
            "Probando cosas en Jupyter... sin link ni hashtag",
            "@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml",
            "¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data",
        ]
    }
)

## Productos y precios
df_productos = pd.DataFrame(
    {
        "producto": [
            "Camisa talla M",
            "Pantalón-XL",
            "Zapato, Talla: 42",
            "Blusa s",
            "Medias 10-12",
            "Polo Talla l",
            "Vestido - 36",
            "Sombrero (talla Única)",
        ],
        "precio": [
            "$1.234,50",
            "USD 45",
            "30,00 €",
            "25.000",
            "$ 0",
            "S/. 120.90",
            "COP 9.990",
            "AR$ 2.550,00",
        ],
    }
)

## Personas, respuestas y direcciones
df_personas = pd.DataFrame(
    {
        "nombre": [
            "ana María LOPEZ",
            "Juan  perez",
            "Ñandú   Gómez",
            "Miguel (Soporte)",
            "  MÓNICA de la CRUZ  ",
            "luis-delgado",
        ],
        "categoria": ["Sí", "si", "SI ", "No", "—", None],
        "direccion": [
            "Calle 45 # 12-34, Bogotá",
            "Av. Siempre Viva 742 - Lima",
            "Cll. 10 No. 5-20 Medellín",
            "Cra 7a # 45-60, Bogotá",
            "Av. 9 #12-34  Cali",
            "Av. Insurgentes Sur 1234, CDMX",
        ],
        "id_raw": [
            "abc-0001",
            "abc 001",
            "ABC0002",
            "Abc_003",
            "abc-00004",
            "ABC-0005",
        ],
    }
)

In [9]:
print("df_texto:")
display(df_texto)
print("\ndf_tweets:")
display(df_tweets)
print("\ndf_productos:")
display(df_productos)
print("\ndf_personas:")
display(df_personas)


df_texto:


,texto
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita...
1,Pandas > numpy? 🤔 Email: persona@example.com
2,Me gusta el café colombiano; es buenísimo!!! #...
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...
4,"TABLAS\t, \n espacios y saltos de líne..."
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300...
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La...
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"



df_tweets:


,tweet
0,RT @maria: Nuevo post en el blog -> http://blo...
1,¡Me encanta Pandas! #datos #Python https://exa...
2,Probando cosas en Jupyter... sin link ni hashtag
3,@juan y @ana lanzaron curso de NLP en https://...
4,¿Pandas o Polars? debátanlo aquí 👉 https://for...



df_productos:


,producto,precio
0,Camisa talla M,"$1.234,50"
1,Pantalón-XL,USD 45
2,"Zapato, Talla: 42","30,00 €"
3,Blusa s,25.000
4,Medias 10-12,$ 0
5,Polo Talla l,S/. 120.90
6,Vestido - 36,COP 9.990
7,Sombrero (talla Única),"AR$ 2.550,00"



df_personas:


,nombre,categoria,direccion,id_raw
0,ana María LOPEZ,Sí,"Calle 45 # 12-34, Bogotá",abc-0001
1,Juan perez,si,Av. Siempre Viva 742 - Lima,abc 001
2,Ñandú Gómez,SI,Cll. 10 No. 5-20 Medellín,ABC0002
3,Miguel (Soporte),No,"Cra 7a # 45-60, Bogotá",Abc_003
4,MÓNICA de la CRUZ,—,Av. 9 #12-34 Cali,abc-00004
5,luis-delgado,None,"Av. Insurgentes Sur 1234, CDMX",ABC-0005


## Inspección rápida

- Explora los cuatro DataFrames
- Identifica qué columnas requieren **limpieza textual** y por qué (espacios, acentos, emojis, URLs, etc.).
- Escribe un breve comentario (como comentario en la celda) con tus observaciones.

In [10]:
# Copie su código aca
df_texto.info()
df_tweets.info()
df_personas.info()
df_productos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texto   8 non-null      object
dtypes: object(1)
memory usage: 196.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   5 non-null      object
dtypes: object(1)
memory usage: 172.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nombre     6 non-null      object
 1   categoria  5 non-null      object
 2   direccion  6 non-null      object
 3   id_raw     6 non-null      object
dtypes: object(4)
memory usage: 324.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column    

## 1) Normaliza a minúsculas

En el DataFrame `df_texto`, crea una nueva columna `texto_min` que contenga el texto de la columna `texto` en minúsculas.

In [11]:
# 1) Normaliza a minúsculas
# Por qué: trabajar en minúsculas reduce la variación por mayúsculas/minúsculas
# y facilita comparaciones y búsquedas. Convertimos a str por si hay NaN o tipos extraños.
df_texto["texto_min"] = df_texto["texto"].astype(str).str.lower()
# Mostrar las columnas originales y transformadas (preview)
df_texto[["texto", "texto_min"]].head()

,texto,texto_min
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita...,¡hola mundo!!! esto es un ejemplo: visita...
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas > numpy? 🤔 email: persona@example.com
2,Me gusta el café colombiano; es buenísimo!!! #...,me gusta el café colombiano; es buenísimo!!! #...
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...,oferta!!! 3x2 en jabón líquido 500ml - código:...
4,"TABLAS\t, \n espacios y saltos de líne...","tablas\t, \n espacios y saltos de líne..."


## 2) Eliminar Espacios

Genera  la columna `texto_espacios` eliminando espacios al inicio/fin

In [12]:
# Copie su código aca
df_texto["texto_espacios"] = df_texto["texto_min"].str.strip()
df_texto[["texto_espacios", "texto_min"]]

,texto_espacios,texto_min
0,¡hola mundo!!! esto es un ejemplo: visita h...,¡hola mundo!!! esto es un ejemplo: visita...
1,pandas > numpy? 🤔 email: persona@example.com,pandas > numpy? 🤔 email: persona@example.com
2,me gusta el café colombiano; es buenísimo!!! #...,me gusta el café colombiano; es buenísimo!!! #...
3,oferta!!! 3x2 en jabón líquido 500ml - código:...,oferta!!! 3x2 en jabón líquido 500ml - código:...
4,"tablas\t, \n espacios y saltos de línea.","tablas\t, \n espacios y saltos de líne..."
5,teléfono: (57) 300-123-45-67; whatsapp +57 300...,teléfono: (57) 300-123-45-67; whatsapp +57 300...
6,dirección: cll 10 # 5-20; medellín. barrio: la...,dirección: cll 10 # 5-20; medellín. barrio: la...
7,"emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…","emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"


,texto_espacios,texto_min
0,¡hola mundo!!! esto es un ejemplo: visita https://miweb.com #datascience :),¡hola mundo!!! esto es un ejemplo: visita https://miweb.com #datascience :)
1,pandas > numpy? 🤔 email: persona@example.com,pandas > numpy? 🤔 email: persona@example.com
2,me gusta el café colombiano; es buenísimo!!! #café #colombia @juan,me gusta el café colombiano; es buenísimo!!! #café #colombia @juan
3,oferta!!! 3x2 en jabón líquido 500ml - código: a-123,oferta!!! 3x2 en jabón líquido 500ml - código: a-123
4,"tablas\t, \n espacios y saltos de línea.","tablas\t, \n espacios y saltos de línea.\r\n"
5,teléfono: (57) 300-123-45-67; whatsapp +57 300 222 33 44,teléfono: (57) 300-123-45-67; whatsapp +57 300 222 33 44
6,dirección: cll 10 # 5-20; medellín. barrio: la américa,dirección: cll 10 # 5-20; medellín. barrio: la américa
7,"emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…","emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"


## 3) Puntuación

Crea `texto_sin_punct` removiendo puntuación y símbolos, conservando letras y espacios.

In [13]:
# Copie su código aca
df_texto["texto_sin_punct"] = df_texto["texto_espacios"].str.replace(
    r"[^\w\s]", "", regex=True
)
df_texto[["texto_sin_punct", "texto_espacios"]]

,texto_sin_punct,texto_espacios
0,hola mundo esto es un ejemplo visita httpsm...,¡hola mundo!!! esto es un ejemplo: visita h...
1,pandas numpy email personaexamplecom,pandas > numpy? 🤔 email: persona@example.com
2,me gusta el café colombiano es buenísimo café ...,me gusta el café colombiano; es buenísimo!!! #...
3,oferta 3x2 en jabón líquido 500ml código a123,oferta!!! 3x2 en jabón líquido 500ml - código:...
4,tablas\t \n espacios y saltos de línea,"tablas\t, \n espacios y saltos de línea."
5,teléfono 57 3001234567 whatsapp 57 300 222 33 44,teléfono: (57) 300-123-45-67; whatsapp +57 300...
6,dirección cll 10 520 medellín barrio la américa,dirección: cll 10 # 5-20; medellín. barrio: la...
7,emoji test símbolos y otros,"emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…"


## 4) Acentos

Elimina los acentos de las vocales del texto `df_texto` y grábalo en una nueva columna `texto_sin_acentos`.

In [15]:
# Copie su código aca
import unicodedata
import pandas as pd


def quitar_acentos(texto):
    if pd.isna(texto):
        return ""
    texto_normalizado = unicodedata.normalize("NFKD", str(texto))
    texto_sin_acentos = "".join(
        c for c in texto_normalizado if not unicodedata.combining(c)
    )
    return texto_sin_acentos


columna_texto = "texto"
df_texto["texto_sin_acentos"] = df_texto[columna_texto].apply(quitar_acentos)


print("Comparación de texto original vs sin acentos:")
display(df_texto[[columna_texto, "texto_sin_acentos"]].head(10))


Comparación de texto original vs sin acentos:


,texto,texto_sin_acentos
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita...,¡Hola Mundo!!! Esto es un EJEMPLO: visita...
1,Pandas > numpy? 🤔 Email: persona@example.com,Pandas > numpy? 🤔 Email: persona@example.com
2,Me gusta el café colombiano; es buenísimo!!! #...,Me gusta el cafe colombiano; es buenisimo!!! #...
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...,Oferta!!! 3x2 en jabon liquido 500ml - CODIGO:...
4,"TABLAS\t, \n espacios y saltos de líne...","TABLAS\t, \n espacios y saltos de line..."
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300...,Telefono: (57) 300-123-45-67; Whatsapp +57 300...
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La...,Direccion: Cll 10 # 5-20; Medellin. Barrio: La...
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…","Emoji test: 😀🙌🏽🏳️‍🌈, simbolos ©®TM y otros..."


0    hola mundo  esto es   un ejemplo visita httpsmiwebcom  datascience  
1                                 pandas  numpy   email personaexamplecom
2             me gusta el cafe colombiano es buenisimo cafe colombia juan
3                          oferta 3x2 en jabon liquido 500ml  codigo a123
4                                tablas\t \n espacios   y saltos de linea
5                        telefono 57 3001234567 whatsapp 57 300 222 33 44
6                        direccion cll 10  520 medellin barrio la america
7                                           emoji test  simbolos  y otros
Name: texto_sin_acentos, dtype: object

## 5) Emojis

Crear la columna `texto_sin_emoji` quitando emojis, pictogramas y símbolos gráficos comunes.

In [16]:
df_texto.columns


Index(['texto', 'texto_min', 'texto_espacios', 'texto_sin_punct',
       'texto_sin_acentos'],
      dtype='object')

In [17]:
import re
import pandas as pd

# Verificamos que el DataFrame exista
try:
    print("Columnas en df_texto:", df_texto.columns.tolist())
except NameError:
    raise NameError(
        "❌ El DataFrame 'df_texto' no está definido. Ejecuta primero el punto donde se crea."
    )

# Usaremos como base la última columna de limpieza disponible
columna_base = "texto_sin_punct"  # tu última versión del texto limpio


# Función para quitar emojis y símbolos
def quitar_emojis(texto):
    if pd.isna(texto):
        return ""
    texto = str(texto)
    emoji_pattern = re.compile(
        "["
        "\U0001f600-\U0001f64f"  # emoticonos
        "\U0001f300-\U0001f5ff"  # símbolos y pictogramas
        "\U0001f680-\U0001f6ff"  # transporte y mapas
        "\U0001f1e0-\U0001f1ff"  # banderas
        "\U00002700-\U000027bf"  # dingbats
        "\U0001f900-\U0001f9ff"  # pictogramas suplementarios
        "\U00002600-\U000026ff"  # símbolos misceláneos
        "]+",
        flags=re.UNICODE,
    )
    texto_sin_emojis = emoji_pattern.sub("", texto)
    return texto_sin_emojis


# Aplicamos la función sobre la columna base
df_texto["texto_sin_emoji"] = df_texto[columna_base].apply(quitar_emojis)

# Mostramos comparación
print("Comparación entre texto con y sin emojis:")
display(df_texto[[columna_base, "texto_sin_emoji"]].head(10))


Columnas en df_texto: ['texto', 'texto_min', 'texto_espacios', 'texto_sin_punct', 'texto_sin_acentos']
Comparación entre texto con y sin emojis:


,texto_sin_punct,texto_sin_emoji
0,hola mundo esto es un ejemplo visita httpsm...,hola mundo esto es un ejemplo visita httpsm...
1,pandas numpy email personaexamplecom,pandas numpy email personaexamplecom
2,me gusta el café colombiano es buenísimo café ...,me gusta el café colombiano es buenísimo café ...
3,oferta 3x2 en jabón líquido 500ml código a123,oferta 3x2 en jabón líquido 500ml código a123
4,tablas\t \n espacios y saltos de línea,tablas\t \n espacios y saltos de línea
5,teléfono 57 3001234567 whatsapp 57 300 222 33 44,teléfono 57 3001234567 whatsapp 57 300 222 33 44
6,dirección cll 10 520 medellín barrio la américa,dirección cll 10 520 medellín barrio la américa
7,emoji test símbolos y otros,emoji test símbolos y otros


## 6) Extrae emails y URLs

En `df_texto`, crea las columnas `email` y `urls` extrayendo emails y URLs del texto original.

**Patrones sugeridos:**
- Email: `[A-Za-z0-9_.+-]+@[A-Za-z0-9-]+\.[A-Za-z0-9.-]+`
- URL: `https?://\S+`

In [18]:
# Copie su código aca
email_pat = r"[A-Za-z0-9_.+-]+@[A-Za-z0-9-]+\.[A-Za-z0-9.-]+"
url_pat = r"https?://\S+"
df_texto["email"] = df_texto["texto"].str.findall(email_pat)
df_texto["urls"] = df_texto["texto"].str.findall(url_pat)

df_texto[["texto", "email", "urls"]]

,texto,email,urls
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita...,[],[https://miweb.com]
1,Pandas > numpy? 🤔 Email: persona@example.com,[persona@example.com],[]
2,Me gusta el café colombiano; es buenísimo!!! #...,[],[]
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...,[],[]
4,"TABLAS\t, \n espacios y saltos de líne...",[],[]
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300...,[],[]
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La...,[],[]
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",[],[]


## 7) Hashtags, menciones y URLs en tweets

En `df_tweets`, crea las columnas `hashtags`, `mentions` y `urls`
extrayendo hashtags, menciones y URLs del texto del tweet.

In [19]:
# Copie su código aca
import re
import pandas as pd

# Primero verificamos que el DataFrame existe
try:
    print("Columnas en df_tweets:", df_tweets.columns.tolist())
except NameError:
    raise NameError(
        "❌ El DataFrame 'df_tweets' no está definido aún. Asegúrate de ejecutarlo antes de este punto."
    )

# Verificamos el nombre de la columna de texto (ajusta si es distinto)
columna_texto = (
    "tweet"  # cámbialo si tu DataFrame usa otro nombre, como 'texto', 'text', etc.
)

if columna_texto not in df_tweets.columns:
    raise KeyError(f"No se encontró la columna '{columna_texto}' en df_tweets.")


# Funciones auxiliares para extraer con expresiones regulares
def extraer_hashtags(texto):
    return re.findall(r"#\w+", str(texto))


def extraer_mentions(texto):
    return re.findall(r"@\w+", str(texto))


def extraer_urls(texto):
    return re.findall(r"http\S+|www\.\S+", str(texto))


# Crear nuevas columnas con las listas extraídas
df_tweets["hashtags"] = df_tweets[columna_texto].apply(extraer_hashtags)
df_tweets["mentions"] = df_tweets[columna_texto].apply(extraer_mentions)
df_tweets["urls"] = df_tweets[columna_texto].apply(extraer_urls)

# Mostramos una vista de ejemplo
print("Ejemplo de extracción en tweets:")
display(df_tweets[[columna_texto, "hashtags", "mentions", "urls"]].head(10))

Columnas en df_tweets: ['tweet']
Ejemplo de extracción en tweets:


,tweet,hashtags,mentions,urls
0,RT @maria: Nuevo post en el blog -> http://blo...,"[#nlp, #python]",[@maria],[http://blog.com/post?id=45]
1,¡Me encanta Pandas! #datos #Python https://exa...,"[#datos, #Python]",[@data_science],[https://example.org]
2,Probando cosas en Jupyter... sin link ni hashtag,[],[],[]
3,@juan y @ana lanzaron curso de NLP en https://...,"[#nlp, #ml]","[@juan, @ana]",[https://cursos.ai]
4,¿Pandas o Polars? debátanlo aquí 👉 https://for...,[#data],[],[https://foro.com]


,tweet,hashtags,mentions,urls
0,RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python,"[#nlp, #python]",[@maria],[http://blog.com/post?id=45]
1,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊,"[#datos, #Python]",[@data_science],[https://example.org]
2,Probando cosas en Jupyter... sin link ni hashtag,[],[],[]
3,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml,"[#nlp, #ml]","[@juan, @ana]",[https://cursos.ai]
4,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data,[#data],[],[https://foro.com]


## 8) ¿Es retuit? Limpia el prefijo

Crea la columna `es_rt` (booleano) si el tweet comienza con `RT`. Luego genera la columna `tweet_sin_rt` quitando el prefijo `RT @usuario:` del inicio.

In [20]:
# Copie su código aca
df_tweets["es_rt"] = df_tweets["tweet"].str.startswith("RT ")
df_tweets["tweet_sin_rt"] = df_tweets["tweet"].str.replace(
    r"^RT\s+@\w+:\s*", "", regex=True
)

df_tweets

,tweet,hashtags,mentions,urls,es_rt,tweet_sin_rt
0,RT @maria: Nuevo post en el blog -> http://blo...,"[#nlp, #python]",[@maria],[http://blog.com/post?id=45],True,Nuevo post en el blog -> http://blog.com/post?...
1,¡Me encanta Pandas! #datos #Python https://exa...,"[#datos, #Python]",[@data_science],[https://example.org],False,¡Me encanta Pandas! #datos #Python https://exa...
2,Probando cosas en Jupyter... sin link ni hashtag,[],[],[],False,Probando cosas en Jupyter... sin link ni hashtag
3,@juan y @ana lanzaron curso de NLP en https://...,"[#nlp, #ml]","[@juan, @ana]",[https://cursos.ai],False,@juan y @ana lanzaron curso de NLP en https://...
4,¿Pandas o Polars? debátanlo aquí 👉 https://for...,[#data],[],[https://foro.com],False,¿Pandas o Polars? debátanlo aquí 👉 https://for...


,tweet,hashtags,mentions,urls,es_rt,tweet_sin_rt
0,RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python,"[#nlp, #python]",[@maria],[http://blog.com/post?id=45],True,Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python
1,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊,"[#datos, #Python]",[@data_science],[https://example.org],False,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊
2,Probando cosas en Jupyter... sin link ni hashtag,[],[],[],False,Probando cosas en Jupyter... sin link ni hashtag
3,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml,"[#nlp, #ml]","[@juan, @ana]",[https://cursos.ai],False,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml
4,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data,[#data],[],[https://foro.com],False,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data


## 9) `tweet_limpio`

Crea `tweet_limpio` removiendo URLs, menciones y hashtags; además, elimina la puntuación sobrante, quita espacios y pasa a minúsculas.

In [21]:
# Copie su código aca
tmp = df_tweets["tweet_sin_rt"] if "tweet_sin_rt" in df_tweets else df_tweets["tweet"]
tmp = tmp.str.replace(r"https?://\S+", " ", regex=True)
tmp = tmp.str.replace(r"@\w+|#\w+", " ", regex=True)
tmp = tmp.str.replace(r"[^\w\sáéíóúÁÉÍÓÚñÑ]", " ", regex=True)
tmp = tmp.str.replace(r"\s+", " ", regex=True).str.strip().str.casefold()
df_tweets["tweet_limpio"] = tmp
df_tweets[["tweet", "tweet_limpio"]]

,tweet,tweet_limpio
0,RT @maria: Nuevo post en el blog -> http://blo...,nuevo post en el blog
1,¡Me encanta Pandas! #datos #Python https://exa...,me encanta pandas
2,Probando cosas en Jupyter... sin link ni hashtag,probando cosas en jupyter sin link ni hashtag
3,@juan y @ana lanzaron curso de NLP en https://...,y lanzaron curso de nlp en
4,¿Pandas o Polars? debátanlo aquí 👉 https://for...,pandas o polars debátanlo aquí


,tweet,tweet_limpio
0,RT @maria: Nuevo post en el blog -> http://blog.com/post?id=45 #nlp #python,nuevo post en el blog
1,¡Me encanta Pandas! #datos #Python https://example.org @data_science 😊,me encanta pandas
2,Probando cosas en Jupyter... sin link ni hashtag,probando cosas en jupyter sin link ni hashtag
3,@juan y @ana lanzaron curso de NLP en https://cursos.ai #nlp #ml,y lanzaron curso de nlp en
4,¿Pandas o Polars? debátanlo aquí 👉 https://foro.com #data,pandas o polars debátanlo aquí


## 10) Talla en `df_productos`

Extrae y estandariza la talla:
- Letras: `XS, S, M, L, XL, XXL`, o `TU` (talla única).
- Números: captura como `talla_num`.
Crea la columna`talla_std` (letras) y `talla_num` (numérica).

In [22]:
# Copie su código aca
pat_letra = r"(?i)\b(xs|s|m|l|xl|xxl|única|unica|u)\b"
pat_num = r"(\d+(?:\.\d+)?)"


def extraer_talla(row):
    txt = str(row["producto"])
    m1 = re.search(pat_letra, txt)
    m2 = re.search(pat_num, txt)
    talla_std = None
    talla_num = None
    if m1:
        val = m1.group(1).lower()
        talla_std = "TU" if val in {"única", "unica", "u"} else val.upper()
    if m2:
        talla_num = float(m2.group(1)) if m2.group(1) is not None else None
    return pd.Series({"talla_std": talla_std, "talla_num": talla_num})


df_productos[["talla_std", "talla_num"]] = df_productos.apply(extraer_talla, axis=1)
df_productos

,producto,precio,talla_std,talla_num
0,Camisa talla M,"$1.234,50",M,NaN
1,Pantalón-XL,USD 45,XL,NaN
2,"Zapato, Talla: 42","30,00 €",NaN,42.0
3,Blusa s,25.000,S,NaN
4,Medias 10-12,$ 0,NaN,10.0
5,Polo Talla l,S/. 120.90,L,NaN
6,Vestido - 36,COP 9.990,NaN,36.0
7,Sombrero (talla Única),"AR$ 2.550,00",TU,NaN


,producto,precio,talla_std,talla_num
0,Camisa talla M,"$1.234,50",M,NaN
1,Pantalón-XL,USD 45,XL,NaN
2,"Zapato, Talla: 42","30,00 €",NaN,42.0
3,Blusa s,25.000,S,NaN
4,Medias 10-12,$ 0,NaN,10.0
5,Polo Talla l,S/. 120.90,L,NaN
6,Vestido - 36,COP 9.990,NaN,36.0
7,Sombrero (talla Única),"AR$ 2.550,00",TU,NaN


## 11) Extraer de precios

Convierte la columna `precio` a numérico (`precio_num`) independiente del formato (`$1.234,50`, `USD 45`, `30,00 €`, `COP 9.990`, etc.).

In [23]:
# Copie su código aca
# ----------------------------------------------------


def parse_precio(s):
    if pd.isna(s):
        return np.nan
    s = str(s).replace("\xa0", " ").strip()
    s2 = re.sub(r"[^0-9,.-]", "", s)
    if "," in s2 and "." in s2:
        s2 = s2.replace(".", "").replace(",", ".")
    elif "," in s2 and "." not in s2:
        s2 = s2.replace(",", ".")
    s2 = s2.replace(",", "")
    try:
        return float(s2)
    except Exception:
        return np.nan


df_productos["precio_num"] = df_productos["precio"].apply(parse_precio)
df_productos

,producto,precio,talla_std,talla_num,precio_num
0,Camisa talla M,"$1.234,50",M,NaN,1234.50
1,Pantalón-XL,USD 45,XL,NaN,45.00
2,"Zapato, Talla: 42","30,00 €",NaN,42.0,30.00
3,Blusa s,25.000,S,NaN,25.00
4,Medias 10-12,$ 0,NaN,10.0,0.00
5,Polo Talla l,S/. 120.90,L,NaN,NaN
6,Vestido - 36,COP 9.990,NaN,36.0,9.99
7,Sombrero (talla Única),"AR$ 2.550,00",TU,NaN,2550.00


,producto,precio,talla_std,talla_num,precio_num
0,Camisa talla M,"$1.234,50",M,NaN,1234.50
1,Pantalón-XL,USD 45,XL,NaN,45.00
2,"Zapato, Talla: 42","30,00 €",NaN,42.0,30.00
3,Blusa s,25.000,S,NaN,25.00
4,Medias 10-12,$ 0,NaN,10.0,0.00
5,Polo Talla l,S/. 120.90,L,NaN,NaN
6,Vestido - 36,COP 9.990,NaN,36.0,9.99
7,Sombrero (talla Única),"AR$ 2.550,00",TU,NaN,2550.00


## 12) Filtrado por texto

Filtra el DataFrame `df_productos` para mostrar solo filas cuyo `producto` contenga **camisa** o **pantalón**, ignorando acentos y mayúsculas/minúsculas.

In [24]:
# Copie su código aca
import unicodedata

columna_producto = "producto"


def quitar_acentos(texto):
    texto = str(texto)
    texto = unicodedata.normalize("NFD", texto)
    texto = texto.encode("ascii", "ignore").decode("utf-8")
    return texto


df_productos["producto_normalizado"] = df_productos[columna_producto].apply(
    lambda x: quitar_acentos(x).lower()
)

filtro = df_productos["producto_normalizado"].str.contains(
    "camisa|pantalon", regex=True
)
df_filtrado = df_productos[filtro]

display(df_filtrado[[columna_producto]])



,producto
0,Camisa talla M
1,Pantalón-XL


In [25]:
mask = (
    df_productos["producto"]
    .str.lower()
    .apply(quitar_acentos)
    .str.contains("camisa|pantalon")
)
df_productos[mask]


,producto,precio,talla_std,talla_num,precio_num,producto_normalizado
0,Camisa talla M,"$1.234,50",M,NaN,1234.5,camisa talla m
1,Pantalón-XL,USD 45,XL,NaN,45.0,pantalon-xl


,producto,precio,talla_std,talla_num,precio_num
0,Camisa talla M,"$1.234,50",M,NaN,1234.5
1,Pantalón-XL,USD 45,XL,NaN,45.0


## 13) Limpieza de nombres propios

En el DataFrame `df_personas`, crea `nombre_limpio` en `df_personas`:
- Quita paréntesis y su contenido.
- Reemplaza guiones por espacio y elimina espacios extra.
- Aplica *title case* y conserva conectores (`de`, `del`, `la`, `y`) en minúscula.

In [26]:
# Copie su código aca
import re
import pandas as pd

# Verificamos que exista el DataFrame y la columna esperada
columna_nombre = "nombre"  # cámbiala si se llama distinto

if columna_nombre not in df_personas.columns:
    raise KeyError(f"La columna '{columna_nombre}' no existe en df_personas.")


# 🔹 Función para limpiar nombres propios
def limpiar_nombre(nombre):
    if pd.isna(nombre):
        return ""
    texto = str(nombre)

    # 1️⃣ Quitar paréntesis y su contenido, por ejemplo: "Juan (Director)" → "Juan"
    texto = re.sub(r"\([^)]*\)", "", texto)

    # 2️⃣ Reemplazar guiones por espacio
    texto = texto.replace("-", " ")

    # 3️⃣ Eliminar espacios duplicados
    texto = re.sub(r"\s+", " ", texto).strip()

    # 4️⃣ Pasar a Title Case (primera letra mayúscula)
    texto = texto.title()

    # 5️⃣ Conectores comunes que deben quedar en minúscula
    conectores = ["De", "Del", "La", "Y", "Las", "Los"]
    for c in conectores:
        texto = re.sub(rf"\b{c}\b", c.lower(), texto)

    return texto


# Aplicar la función
df_personas["nombre_limpio"] = df_personas[columna_nombre].apply(limpiar_nombre)

# Mostrar comparativa
display(df_personas[[columna_nombre, "nombre_limpio"]].head(10))


,nombre,nombre_limpio
0,ana María LOPEZ,Ana María Lopez
1,Juan perez,Juan Perez
2,Ñandú Gómez,Ñandú Gómez
3,Miguel (Soporte),Miguel
4,MÓNICA de la CRUZ,Mónica de la Cruz
5,luis-delgado,Luis Delgado


0      Ana María Lopez
1           Juan Perez
2          Ñandú Gómez
3               Miguel
4    Mónica de la Cruz
5         Luis Delgado
Name: nombre_limpio, dtype: object

## 14) Normaliza respuestas categóricas

En el DataFrame `df_personas`, convierte `categoria` en booleano `categoria_bool` donde cualquier variante de **sí** (con/ sin tilde) sea `True`; el resto `False`.

In [27]:
# Copie su código aca
def a_bool_si(s):
    if s is None:
        return False
    val = str(s).strip().casefold()
    return val in {"si", "sí", "si.", "sí."}


df_personas["categoria_bool"] = df_personas["categoria"].apply(a_bool_si)
df_personas

,nombre,categoria,direccion,id_raw,nombre_limpio,categoria_bool
0,ana María LOPEZ,Sí,"Calle 45 # 12-34, Bogotá",abc-0001,Ana María Lopez,True
1,Juan perez,si,Av. Siempre Viva 742 - Lima,abc 001,Juan Perez,True
2,Ñandú Gómez,SI,Cll. 10 No. 5-20 Medellín,ABC0002,Ñandú Gómez,True
3,Miguel (Soporte),No,"Cra 7a # 45-60, Bogotá",Abc_003,Miguel,False
4,MÓNICA de la CRUZ,—,Av. 9 #12-34 Cali,abc-00004,Mónica de la Cruz,False
5,luis-delgado,None,"Av. Insurgentes Sur 1234, CDMX",ABC-0005,Luis Delgado,False


,nombre,categoria,direccion,id_raw,nombre_limpio,categoria_bool
0,ana María LOPEZ,Sí,"Calle 45 # 12-34, Bogotá",abc-0001,Ana María Lopez,True
1,Juan perez,si,Av. Siempre Viva 742 - Lima,abc 001,Juan Perez,True
2,Ñandú Gómez,SI,Cll. 10 No. 5-20 Medellín,ABC0002,Ñandú Gómez,True
3,Miguel (Soporte),No,"Cra 7a # 45-60, Bogotá",Abc_003,Miguel,False
4,MÓNICA de la CRUZ,—,Av. 9 #12-34 Cali,abc-00004,Mónica de la Cruz,False
5,luis-delgado,None,"Av. Insurgentes Sur 1234, CDMX",ABC-0005,Luis Delgado,False


## 15) Ciudad desde la dirección

En el DataFrame `df_personas`, Extrae la **ciudad** en una columna `ciudad`

In [28]:
# Copie su código aca
import re

# Ajusta el nombre de la columna si se llama diferente
columna_direccion = "direccion"

# Extraemos la ciudad: asumimos que es la última palabra o parte después de una coma
df_personas["ciudad"] = df_personas[columna_direccion].str.extract(r",\s*([^,]+)$")

# Mostramos resultado
display(df_personas[[columna_direccion, "ciudad"]].head(10))

,direccion,ciudad
0,"Calle 45 # 12-34, Bogotá",Bogotá
1,Av. Siempre Viva 742 - Lima,NaN
2,Cll. 10 No. 5-20 Medellín,NaN
3,"Cra 7a # 45-60, Bogotá",Bogotá
4,Av. 9 #12-34 Cali,NaN
5,"Av. Insurgentes Sur 1234, CDMX",CDMX


In [29]:
def extraer_ciudad(s):
    s = str(s)
    m = re.search(r",\s*([^,]+)$", s)
    if m:
        return m.group(1).strip()
    return s.strip().split()[-1]


df_personas["ciudad"] = df_personas["direccion"].apply(extraer_ciudad)
df_personas


,nombre,categoria,direccion,id_raw,nombre_limpio,categoria_bool,ciudad
0,ana María LOPEZ,Sí,"Calle 45 # 12-34, Bogotá",abc-0001,Ana María Lopez,True,Bogotá
1,Juan perez,si,Av. Siempre Viva 742 - Lima,abc 001,Juan Perez,True,Lima
2,Ñandú Gómez,SI,Cll. 10 No. 5-20 Medellín,ABC0002,Ñandú Gómez,True,Medellín
3,Miguel (Soporte),No,"Cra 7a # 45-60, Bogotá",Abc_003,Miguel,False,Bogotá
4,MÓNICA de la CRUZ,—,Av. 9 #12-34 Cali,abc-00004,Mónica de la Cruz,False,Cali
5,luis-delgado,None,"Av. Insurgentes Sur 1234, CDMX",ABC-0005,Luis Delgado,False,CDMX


## 16) Normaliza y valida IDs

En el DataFrame `df_personas`, a partir de `id_raw`, crea la columna `id_norm` con formato `ABC-0001` (tres letras + guion + 4 dígitos) y `id_valido` (True/False).

In [30]:
# Copie su código aca
import re
import pandas as pd

# Asegura que exista la columna base
columna_id = "id_raw"

if columna_id not in df_personas.columns:
    raise KeyError(f"La columna '{columna_id}' no existe en df_personas.")


# 🔹 Función para normalizar el ID
def normalizar_id(valor):
    if pd.isna(valor):
        return ""
    texto = str(valor).upper()
    # Elimina espacios y caracteres no alfanuméricos
    texto = re.sub(r"[^A-Z0-9]", "", texto)

    # Busca 3 letras seguidas de 1–4 dígitos
    match = re.match(r"([A-Z]{3})(\d{1,4})", texto)
    if match:
        letras, numeros = match.groups()
        return f"{letras}-{int(numeros):04d}"
    return ""


# Aplicamos la normalización
df_personas["id_norm"] = df_personas[columna_id].apply(normalizar_id)

# 🔹 Validamos formato correcto (ABC-0001)
df_personas["id_valido"] = df_personas["id_norm"].str.match(r"^[A-Z]{3}-\d{4}$")

# Mostramos resultado
display(df_personas[[columna_id, "id_norm", "id_valido"]].head(10))


,id_raw,id_norm,id_valido
0,abc-0001,ABC-0001,True
1,abc 001,ABC-0001,True
2,ABC0002,ABC-0002,True
3,Abc_003,ABC-0003,True
4,abc-00004,ABC-0000,True
5,ABC-0005,ABC-0005,True


,id_raw,id_norm,id_valido
0,abc-0001,ABC-0001,True
1,abc 001,ABC-0001,True
2,ABC0002,ABC-0002,True
3,Abc_003,ABC-0003,True
4,abc-00004,ABC-0004,True
5,ABC-0005,ABC-0005,True


## 17) Mini *pipeline* de limpieza general

Escribe una función `limpia_basica(s)` que: ponga el texto en minuscula, quite acentos, URLs, menciones y hashtags, puntuación, dígitos y colapse espacios. Aplícala a `df_texto['texto']` y guarda en `texto_limpio_final`.

In [31]:
# Copie su código aca
import re
import unicodedata
import pandas as pd


def limpia_basica(s):
    if pd.isna(s):
        return ""
    texto = str(s)

    # Pasar a minúsculas
    texto = texto.lower()

    # Quitar acentos
    texto = unicodedata.normalize("NFD", texto)
    texto = texto.encode("ascii", "ignore").decode("utf-8")

    # Quitar URLs
    texto = re.sub(r"http\S+|www\.\S+", "", texto)

    # Quitar menciones y hashtags
    texto = re.sub(r"[@#]\w+", "", texto)

    # Quitar puntuación
    texto = re.sub(r"[^\w\s]", " ", texto)

    # Quitar dígitos
    texto = re.sub(r"\d+", "", texto)

    # Colapsar espacios múltiples
    texto = re.sub(r"\s+", " ", texto).strip()

    return texto


# Aplicar al DataFrame
df_texto["texto_limpio_final"] = df_texto["texto"].apply(limpia_basica)

# Mostrar resultado
display(df_texto[["texto", "texto_limpio_final"]].head(10))


,texto,texto_limpio_final
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita...,hola mundo esto es un ejemplo visita
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas numpy email persona com
2,Me gusta el café colombiano; es buenísimo!!! #...,me gusta el cafe colombiano es buenisimo
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...,oferta x en jabon liquido ml codigo a
4,"TABLAS\t, \n espacios y saltos de líne...",tablas espacios y saltos de linea
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300...,telefono whatsapp
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La...,direccion cll medellin barrio la america
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",emoji test simbolos y otros


,texto,texto_limpio_final
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita https://miweb.com #DataScience :),hola mundo esto es un ejemplo visita
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas numpy email persona com
2,Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan,me gusta el cafe colombiano es buenisimo
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123,oferta x en jabon liquido ml codigo a
4,"TABLAS\t, \n espacios y saltos de línea.\r\n",tablas espacios y saltos de linea
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44,telefono whatsapp
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La América,direccion cll medellin barrio la america
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",emoji test simbolos y otros


## 18)¿Qué tanto cambió el texto?

En el dataframe `df_texto`, 

Cual es la diferencia entre la longitud original (`len_raw`) y la longitud tras la limpieza (`len_clean`)?

In [32]:
# Copie su código aca
df_texto["len_raw"] = df_texto["texto"].str.len()
df_texto["len_clean"] = df_texto["texto_limpio_final"].str.len()
df_texto[["texto", "texto_limpio_final", "len_raw", "len_clean"]]

,texto,texto_limpio_final,len_raw,len_clean
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita...,hola mundo esto es un ejemplo visita,84,36
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas numpy email persona com,48,30
2,Me gusta el café colombiano; es buenísimo!!! #...,me gusta el cafe colombiano es buenisimo,66,40
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO:...,oferta x en jabon liquido ml codigo a,52,37
4,"TABLAS\t, \n espacios y saltos de líne...",tablas espacios y saltos de linea,48,33
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300...,telefono whatsapp,56,17
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La...,direccion cll medellin barrio la america,54,40
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",emoji test simbolos y otros,42,27


,texto,texto_limpio_final,len_raw,len_clean
0,¡Hola Mundo!!! Esto es un EJEMPLO: visita https://miweb.com #DataScience :),hola mundo esto es un ejemplo visita,84,36
1,Pandas > numpy? 🤔 Email: persona@example.com,pandas numpy email persona com,48,30
2,Me gusta el café colombiano; es buenísimo!!! #Café #Colombia @juan,me gusta el cafe colombiano es buenisimo,66,40
3,Oferta!!! 3x2 en jabón líquido 500ml - CÓDIGO: A-123,oferta x en jabon liquido ml codigo a,52,37
4,"TABLAS\t, \n espacios y saltos de línea.\r\n",tablas espacios y saltos de linea,48,33
5,Teléfono: (57) 300-123-45-67; Whatsapp +57 300 222 33 44,telefono whatsapp,56,17
6,Dirección: Cll 10 # 5-20; Medellín. Barrio: La América,direccion cll medellin barrio la america,54,40
7,"Emoji test: 😀🙌🏽🏳️‍🌈, símbolos ©®™ y otros…",emoji test simbolos y otros,42,27


In [33]:
!pip install pyarrow


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [34]:
!pip install fastparquet


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [36]:
import pyarrow

print(pyarrow.__version__)


22.0.0


In [39]:
df_texto.to_parquet("archivo.parquet", index=False)
print("Archivo 'archivo.parquet' guardado correctamente.")


Archivo 'archivo.parquet' guardado correctamente.


In [40]:
df_texto.to_csv("archivo.csv", index=False, encoding="utf-8")
print("✅ Archivo guardado correctamente como 'archivo.csv'")


✅ Archivo guardado correctamente como 'archivo.csv'


In [43]:
df_texto.to_parquet("archivo.parquet", index=False)
print("Archivo 'archivo.parquet' guardado correctamente.")

Archivo 'archivo.parquet' guardado correctamente.


### Guardar de resultados

Guarda el dataframe df_texto en parquet con `df_texto.to_parquet('archivo.parquet', index=False)`.

**Phd. Jose R. Zapata**
- [https://joserzapata.github.io/](https://joserzapata.github.io/)
- [https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/](https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/)